## 라이브러리, 페이지 로드

In [ ]:
import pandas as pd
import requests 
import re
from bs4 import BeautifulSoup as bs

In [ ]:
code=192150
page_no=1
review_list=[] #예시:모가디슈

In [ ]:
url=f"https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={page_no}"    
headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}
response=requests.get(url,headers=headers) 
html = bs(response.text,'lxml')
html

## 전체 리뷰 수(total_cnt) & 페이지 수(page_end) 로드 

In [ ]:
total = html.select('div.score_total > strong.total > em') #전체 리뷰 수가 들어있는 태그 크롤링
total_cnt = int(total[0].text.replace(',','')) #숫자만 추출
total_cnt

In [ ]:
page_end = int(total_cnt / 10) #페이지 하나 당 리뷰 10개 
if total_cnt%10 != 0:
    page_end += 1 
page_end

## 네이버 영화관련 최종 크롤링 

In [ ]:
# 페이지 번호, 댓글 리스트를 매개변수로 설정
# 순차적으로 댓글과 여러 정보를 딕셔너리로 저장 후 리스트에 최종으로 담는다. (별다른 리턴값은 없다)
import re

def get_review_list(page_no, review_list):
    """
    페이지별로 10개씩 댓글 및 평점을 수집하여 리스트에 넣는다.
    """
    url=f"https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={page_no}"    
    headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}
    response=requests.get(url,headers=headers) 
    html = bs(response.text,'lxml')
    
        # 댓글(review), 평점(score) 크롤링 태그경로 
    review = html.select('div.score_result > ul > li > div.score_reple > p > span')
    score = html.select('div.score_result > ul > li > div.star_score > em')

    # review에는 관람 여부, 스포일러 여부, 댓글 총 3가지의 데이터가 있다.
    # 상황에 따라 알맞는 데이터를 가져오기 위해 제어변수 num을 설정 및 초기화 하고,
    # 댓글 자체의 순서를 제어할 cnt 변수 생성, 초기화한다.
    num = 0
    cnt = 0
    
        # 공감 비공감 갯수 크롤링 태그경로 
    like = html.select('div.btn_area > a._sympathyButton> strong')
    unlike = html.select('div.btn_area > a._notSympathyButton > strong')  
    
    while (num < len(review)):
        # len(review)는 한 페이지 당 댓글 수 ( 단, 관람객이나 스포일러라면 +1 )
        # 정보를 담을 딕셔너리 초기화 (info_dic)
        info_dic = {"댓글번호":'', "작성일자":'', "댓글":'', "평점":0, "관람여부":0, "스포여부":0, "공감수":0, "비공감수":0, "id":''} 

        # 댓글번호 크롤링 태그경로를 통한 추출
        review_no = str(html.select('div.score_reple > dl > dt > em > a')[cnt]) #댓글번호는 관람객,스포일러 상관없이 아이디 당 존재 
        info_dic["댓글번호"] = int(re.findall('\d+',review_no)[0]) #여기서 0은 인덱싱 고정
        
        # id 및 작성일자 크롤링 태그경로를 통한 추출 
        info_dic["id"] = html.select('div.score_reple > dl > dt ')[cnt].text.strip().split('\n\n\n')[0]
        info_dic["작성일자"] = html.select('div.score_reple > dl > dt ')[cnt].text.strip().split('\n\n\n')[1]
        # 0,1은 리스트 인덱싱
        
        # 댓글 추출 (관람 여부, 스포일러 여부 추가)
        # 관람 여부 확인 -> 스포일러 여부 확인 -> 댓글 저장 순으로 진행한다.
        # 댓글이 저장되면 최종 저장한 인덱스 그 다음으로 num을 이동시킨다.
        if review[num].text == '관람객':
            info_dic["관람여부"] = 1
            if review[num+1].text == '스포일러가 포함된 감상평입니다. 감상평 보기':
                info_dic["스포여부"] = 1
                info_dic["댓글"] = review[num+2].text.strip()
                num += 3
            else :   
                info_dic["댓글"] = review[num+1].text.strip()
                num += 2
        elif review[num].text == '스포일러가 포함된 감상평입니다. 감상평 보기':
            info_dic["스포여부"] = 1
            info_dic["댓글"] = review[num+1].text.strip()
            num += 2
        else :
            info_dic["댓글"] = review[num].text.strip()
            num += 1

        # 평점 추출
        info_dic["평점"] = score[cnt].text.strip()
        
        # 공감, 비공감수 추출
        info_dic["공감수"] = int(re.findall('\d+',str(like[cnt]))[1])
        info_dic["비공감수"] = int(re.findall('\d+',str(unlike[cnt]))[1])            
        
        # 댓글의 모든 정보를 넣었으니 순서를 다음으로 넘긴다.
        cnt += 1
        # 정보가 담긴 딕셔너리를 리스트에 저장
        review_list.append(info_dic)

## 페이지 반복

In [ ]:
# 처음부터 마지막 페이지까지 반복
while (page_no <= 20):
    # get_review_list 함수를 통해 댓글 정보들을 리스트에 담는다
    get_review_list(page_no, review_list)
    page_no += 1

## 데이터 프레임 생성

In [ ]:
review_final = pd.DataFrame(review_list)

In [ ]:
review_final['작성일자']=pd.to_datetime(review_final['작성일자'])

In [ ]:
review_final['평점']=review_final['평점'].astype(int)

In [ ]:
review_final